In [158]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import matplotlib as plt
from sklearn.datasets import make_blobs
from tensorflow import keras

In [159]:
# Load Data
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [160]:
# Create the target
y_var = charity_df.IS_SUCCESSFUL
y_var.head()

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [161]:
# What variable(s) are considered to be the features and what variable are neither for your model?
X = charity_df.drop(columns=['IS_SUCCESSFUL', 'NAME'])
X.head()

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692
4,10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590


In [162]:
# Using the methods described in this module, preprocess all numerical and categorical variables, as needed:
string_dtypes = X.dtypes[X.dtypes == "object"].index.tolist()
X[string_dtypes].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [163]:
class_count = X.CLASSIFICATION.value_counts()
class_count.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [164]:
# Determine which values to replace
replace_class = list(class_count[class_count < 4000].index)

In [165]:
# Replace in DataFrame
for classification in replace_class:
    X.CLASSIFICATION = X.CLASSIFICATION.replace(classification,"Other")

In [166]:
X.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [167]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [168]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(X[string_dtypes]))

In [169]:
# Add encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(string_dtypes)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [170]:
# Merge the two DataFrames together
charity_new_df = pd.merge(X, encode_df, left_index=True, right_index=True).drop(string_dtypes, 1)
charity_new_df.head()

,EIN,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [171]:
# Split our preprocessed data into our features and target arrays
y = y_var
X = charity_new_df

In [172]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [173]:
# Standardize numerical variables using TensorFlow’s StandardScaler class
scaler = StandardScaler()

In [174]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [175]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [176]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

In [177]:
# Train the model
log_classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [178]:
# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f"Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

Logistic regression model accuracy: 0.534


In [179]:
# Define hidden layers
number_input_features = len(X_train_scaled[0])
hidden_layer1 =  8
hidden_layer2 = 5

In [180]:
nn = tf.keras.models.Sequential()

In [181]:
# Hidden layer 1
nn.add(tf.keras.layers.Dense(units=hidden_layer1, input_dim=number_input_features, activation="relu"))

In [182]:
# Hidden layer 2
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

In [183]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [184]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [185]:
# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 8)                 408       
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 459
Trainable params: 459
Non-trainable params: 0
_________________________________________________________________


In [186]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train.values, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 117us/sample - loss: 0.6005 - accuracy: 0.7039
Epoch 2/100
25724/25724 [==============================] - 2s 77us/sample - loss: 0.5669 - accuracy: 0.7231
Epoch 3/100
25724/25724 [==============================] - 2s 79us/sample - loss: 0.5615 - accuracy: 0.7249
Epoch 4/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5590 - accuracy: 0.7261
Epoch 5/100
25724/25724 [==============================] - 2s 84us/sample - loss: 0.5574 - accuracy: 0.7283
Epoch 6/100
25724/25724 [==============================] - 2s 81us/sample - loss: 0.5561 - accuracy: 0.7280
Epoch 7/100
25724/25724 [==============================] - 2s 81us/sample - loss: 0.5552 - accuracy: 0.7288
Epoch 8/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5540 - accuracy: 0.7297
Epoch 9/100
25724/25724 [==============================] - 2s 88us/sample - loss: 0.5532 - accuracy: 0.7301
Epoc

In [187]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test.values, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5671 - accuracy: 0.7247
Loss: 0.5585784084942876, Accuracy: 0.7246647477149963
